In [380]:
import pandas as pd
import datetime

import progressbar
import time

from elasticsearch import Elasticsearch
from pandas.io.json import json_normalize

def to_date(s):
    dt = datetime.datetime.fromtimestamp(s / 1e3)
    return dt.strftime("%Y-%m-%d")

dsl_id = {
        'query': {'bool': {'must': [
                                    {'terms': {'order_type': ['vip_scan']}},
                                   # {'terms': {'shop_id': ['10069']}},
                                    {'query_string': {'query': 'skutotal_price != 0'}},
                                    {'terms': {'order_status': ['confirmed',
                                                                'done',
                                                                'fulfilled']}},
                                    
                                    ]}},
        "aggs": {
            "my_buckets": {
                "composite": {
                    'size': 9999999,
                    "sources": [
                       # {"日期": {"date_histogram": {"field": "paytime", "interval": "1d"}}},
                       # {"单价": {"terms": {"field": 'skutotal_price'}}},
                        {"门店": {"terms": {"field": 'shop_id'}}}
                        
                        ]
                },
                #'aggregations': {#'平均单价': {'avg': {'field': 'skutotal_price'}},
                                 #'最大金额':{'max':{'field':'skutotal_price'}},
                                # '数据':{'extended_stats':{'field':'skutotal_price'}},
                                 #'订单数':{'cardinality':{'field':'order_id'}},
                                 #'n天订单数': {'sum': {'field': 'skutotal_price'}}   }
             
            }
        }
    }
es_id = Elasticsearch(['http://172.18.198.27:9200/'], timeout=30)
res_id = es_id.search(index='es_order_table', doc_type='order', body=dsl_id,
                _source=False, explain=False, track_scores=False, track_total_hits=False, size=0,
                search_type='query_then_fetch')
df_id = json_normalize(res_id['aggregations']['my_buckets']['buckets'])
df_id.columns = map(lambda x: x.split('key.')[-1] if x.startswith('key.') else x, list(df_id.columns))
df_id.head()  #门店所有id

,doc_count,门店
0,11,358
1,5930,365
2,6004,498
3,3375,501
4,1,504


In [381]:
#df_id.to_csv('门店列表.csv')

In [364]:
order_avg,order_max,order_std,order_yssd=[],[],[],[]
bar = progressbar

for i in bar.progressbar(range(len(df_id))):
    time.sleep(0.1)
   # bar.update(i)
    
    shop_list=[]
    shop_list.append(int(df_id.iloc[i]['门店']))
    dsl_sj = {
        'query': {'bool': {'must': [
                                    {'terms': {'order_type': ['vip_scan']}},
                                    {'terms': {'shop_id': shop_list}},
                                    {'query_string': {'query': 'skutotal_price != 0'}},
                                    {'terms': {'order_status': ['confirmed',
                                                                'done',
                                                                'fulfilled']}},
                                    
                                    ]}},
            "aggs": {
                "my_buckets": {
                    "composite": {
                        'size': 9999999,
                        "sources": [
                           # {"日期": {"date_histogram": {"field": "paytime", "interval": "1d"}}},
                            {"单价": {"terms": {"field": 'skutotal_price'}}},
                            {"订单": {"terms": {"field": 'order_id'}}}
                            ]
                },
                #'aggregations': {#'平均单价': {'avg': {'field': 'skutotal_price'}},
                                 #'最大金额':{'max':{'field':'skutotal_price'}},
                                # '数据':{'extended_stats':{'field':'skutotal_price'}},
                                 #'订单数':{'cardinality':{'field':'order_id'}},
                                 #'n天订单数': {'sum': {'field': 'skutotal_price'}}   }
             
                }
            }
    }
    es_sj = Elasticsearch(['http://172.18.198.27:9200/'], timeout=30)
    res_sj = es_sj.search(index='es_order_table', doc_type='order', body=dsl_sj,
                        _source=False, explain=False, track_scores=False, track_total_hits=False, size=0,
                        search_type='query_then_fetch')
    df_sj = json_normalize(res_sj['aggregations']['my_buckets']['buckets'])
    df_sj.columns = map(lambda x: x.split('key.')[-1] if x.startswith('key.') else x, list(df_sj.columns))
    #list化所有订单
    
    shop_order_1,shop_order_2,shop_order_3 = [],[],[]

    for i in range(len(df_sj)):
        if df_sj.iloc[i]['单价']<1:   #过小订单
            continue
        else:
            shop_order_1.append(df_sj.iloc[i]['单价'])

    #除去异常值后的订单价格列表   shop_order 


    large_number = pd.Series(shop_order_1).quantile(0.9)

    #搞出一个较大值

    for i in range(len(shop_order_1)):
        if shop_order_1[i] > large_number:
            if int(shop_order_1[i])/100 == int(shop_order_1[i]/100):
                shop_order_3.append(shop_order_1[i])
        else:
            shop_order_2.append(shop_order_1[i])
    
    if shop_order_2 == []:
        shop_order_2.append(0)
    if shop_order_3==[]:
        shop_order_3.append(0)
        
    order_avg.append(pd.Series(shop_order_2).mean())
    order_max.append(pd.Series(shop_order_2).max())
    order_std.append(pd.Series(shop_order_2).std())
    order_yssd.append(shop_order_3)

100% (754 of 754) |######################| Elapsed Time: 0:08:49 Time:  0:08:49


754

In [366]:
df_id['平均值'] = order_avg
df_id['最大值'] = order_max
df_id['标准差'] = order_std
df_id['疑似刷单价格'] = order_yssd


#df_id

,doc_count,门店,平均值,最大值,标准差,疑似刷单价格
0,11,358,23.181818,30.000000,8.553574,[0]
1,5897,365,16.925844,50.000000,11.463946,"[100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100..."
2,5984,498,14.964954,44.849998,9.566938,"[100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100..."
3,3358,501,6.609937,16.500000,3.833244,[100.0]
4,1,504,25.900000,25.900000,NaN,[0]
5,17,525,13.113333,30.570000,9.005036,[0]
6,87,546,19.432308,53.009998,12.226111,[0]
7,465,562,6.413046,14.500000,2.897289,[0]
8,2,9997,6.500000,6.500000,NaN,[0]
9,86,10022,0.000000,0.000000,NaN,[0]


In [367]:
df_id.fillna(0)

,doc_count,门店,平均值,最大值,标准差,疑似刷单价格
0,11,358,23.181818,30.000000,8.553574,[0]
1,5897,365,16.925844,50.000000,11.463946,"[100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100..."
2,5984,498,14.964954,44.849998,9.566938,"[100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100..."
3,3358,501,6.609937,16.500000,3.833244,[100.0]
4,1,504,25.900000,25.900000,0.000000,[0]
5,17,525,13.113333,30.570000,9.005036,[0]
6,87,546,19.432308,53.009998,12.226111,[0]
7,465,562,6.413046,14.500000,2.897289,[0]
8,2,9997,6.500000,6.500000,0.000000,[0]
9,86,10022,0.000000,0.000000,0.000000,[0]


In [378]:
df_id.fillna(0).to_csv('商家套现数据_new.csv')

In [373]:
taoxian_shop_list,hao_shop_list=[],[]#有过疑似套现商家列表和好商家列表
    

for i in bar.progressbar(range(len(df_id))):
    time.sleep(0.1)
    

    if df_id.iloc[i]['疑似刷单价格'] != [0]:
        taoxian_shop_list.append(df_id.iloc[i]['门店'])
    else:
        hao_shop_list.append(df_id.iloc[i]['门店'])

100% (754 of 754) |######################| Elapsed Time: 0:01:17 Time:  0:01:17


In [375]:
len(taoxian_shop_list)

127